In [ ]:
# 1) Установка (bash-ячейка)
!pip install -q ultralytics opencv-python-headless


project/
├── dataset/
│   ├── images/
│   │   ├── train/
│   │   │   ├── img1.jpg
│   │   │   └── ...
│   │   ├── val/
│   │   │   ├── img2.jpg
│   │   │   └── ...
│   └── labels/
│       ├── train/
│       │   ├── img1.txt
│       │   └── ...
│       ├── val/
│       │   ├── img2.txt
│       │   └── ...
├── data.yaml


class x_center y_center width height
(annotation)
0 0.5 0.5 0.2 0.3
1 0.7 0.8 0.1 0.1
(normalized data)

In [ ]:
# 2) Импорты и проверка GPU
import os
from ultralytics import YOLO
import pandas as pd
import cv2
import glob

print("CUDA:", YOLO._get_device())  # проверка доступности GPU


In [ ]:
# 3) Подготовка data.yaml (раз в ноутбуке)
# Предположим, что данные в /kaggle/input/my_dataset
data_yaml = """
path: /kaggle/working/data      # корневой каталог (для копирования/размещения)
train: images/train
val:   images/val
nc:  3                           # число классов
names: ['class1','class2','class3']
"""
with open('data.yaml','w') as f:
    f.write(data_yaml)
print(open('data.yaml').read())


In [ ]:
# 4) (Опционально) Скопировать данные во временный рабочий каталог
!mkdir -p data/images/train data/images/val
!cp -r /kaggle/input/my_dataset/train/images/* data/images/train/
!cp -r /kaggle/input/my_dataset/valid/images/* data/images/val/
!cp -r /kaggle/input/my_dataset/train/labels/* data/labels/train/
!cp -r /kaggle/input/my_dataset/valid/labels/* data/labels/val/


In [ ]:
# 5) Обучение
model = YOLO('yolov8n.pt')  # или 'yolov8s.pt', 'yolov8m.pt' и т.д.
results = model.train(
    data='data.yaml',
    epochs=30,
    imgsz=640,
    batch=16,
    lr0=1e-3,
    project='yolo8_run',   # папка с логами и весами
    name='exp1',
    patience=5
)
print("Best model saved to:", results.path / 'weights' / 'best.pt')


In [ ]:
# 6) Инференс + сбор сабмита
best = 'yolo8_run/exp1/weights/best.pt'
model = YOLO(best)

records = []
for img_path in glob.glob('/kaggle/input/my_dataset/test/images/*.jpg'):
    res = model(img_path, conf=0.25, iou=0.5)[0]
    for box in res.boxes:
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        conf = float(box.conf)
        cls  = int(box.cls)
        records.append([os.path.basename(img_path), cls, conf, x1, y1, x2, y2])

sub = pd.DataFrame(records, columns=[
    'image','class','confidence','x1','y1','x2','y2'])
sub.to_csv('submission.csv', index=False)
print("Submission saved:", sub.shape)


[x1, y1, x2, y2, confidence, class_id]
output